<a href="https://colab.research.google.com/github/fangyiyu/Fangyi_Yu_Queens/blob/master/Mnist_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!wget -O comp551w18-modified-mnist.zip https://www.dropbox.com/sh/dhu4cu8l8e32cvl/AACSbGE9X6P7an61STwwr8R0a?dl=0

--2020-09-28 18:15:33--  https://www.dropbox.com/sh/dhu4cu8l8e32cvl/AACSbGE9X6P7an61STwwr8R0a?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.1, 2620:100:6018:1::a27d:301
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /sh/raw/dhu4cu8l8e32cvl/AACSbGE9X6P7an61STwwr8R0a [following]
--2020-09-28 18:15:33--  https://www.dropbox.com/sh/raw/dhu4cu8l8e32cvl/AACSbGE9X6P7an61STwwr8R0a
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc4d53dfc82e63385a0222bfa51c.dl.dropboxusercontent.com/zip_download_get/AjrnSOGMOjt5erP4_ZNO87siKC6tzu78rZG76VItF3p6bwaqphyKBKLC3Q1a-CyDCTWkATfV40aCRl74xgtwxJbGeI3X8jMKpPHNv-8Oo2p_Ig [following]
--2020-09-28 18:15:34--  https://uc4d53dfc82e63385a0222bfa51c.dl.dropboxusercontent.com/zip_download_get/AjrnSOGMOjt5erP4_ZNO87siKC6tzu78rZG76VItF3p6bwaqphyKBKLC3Q1a-CyDCTWkATfV40aCRl74xgtw

In [6]:
!unzip comp551w18-modified-mnist.zip

Archive:  comp551w18-modified-mnist.zip
  inflating: test_x.csv              
  inflating: train_y.csv             
  inflating: train_x.csv             


In [7]:
from __future__ import absolute_import, division, print_function
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split

In [17]:
from tensorflow.keras import Model, layers
# MNIST dataset parameters.
num_classes = 10 # 0 to 9 digits

# Training parameters.
learning_rate = 0.001
training_steps = 200
batch_size = 128
display_step = 10

# Network parameters.
conv1_filters = 32 # number of filters for 1st conv layer.
conv2_filters = 64 # number of filters for 2nd conv layer.
fc1_units = 1024 # number of neurons for 1st fully-connected layer.

In [18]:
x_ = np.loadtxt("train_x.csv", delimiter=",")# load from text
y_ = np.loadtxt("train_y.csv", delimiter=",")
x_train, x_test, y_train, y_test = train_test_split(x_,y_,test_size=0.2, random_state = 17)

In [19]:
# Convert to float32.
x_train, x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)

# Normalize images value from [0, 255] to [0, 1].
x_train, x_test = x_train / 255., x_test / 255.

In [20]:
(x_train.shape,x_test.shape,y_train.shape,y_test.shape)

((40000, 4096), (10000, 4096), (40000,), (10000,))

In [21]:
# Use tf.data API to shuffle and batch data.
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)

In [22]:
# Create TF Model.
class ConvNet(Model):
    # Set layers.
    def __init__(self):
        super(ConvNet, self).__init__()
        # Convolution Layer with 32 filters and a kernel size of 5.
        self.conv1 = layers.Conv2D(32, kernel_size=5, activation=tf.nn.relu)
        # Max Pooling (down-sampling) with kernel size of 2 and strides of 2. 
        self.maxpool1 = layers.MaxPool2D(2, strides=2)

        # Convolution Layer with 64 filters and a kernel size of 3.
        self.conv2 = layers.Conv2D(64, kernel_size=3, activation=tf.nn.relu)
        # Max Pooling (down-sampling) with kernel size of 2 and strides of 2. 
        self.maxpool2 = layers.MaxPool2D(2, strides=2)

        # Flatten the data to a 1-D vector for the fully connected layer.
        self.flatten = layers.Flatten()

        # Fully connected layer.
        self.fc1 = layers.Dense(1024)
        # Apply Dropout (if is_training is False, dropout is not applied).
        self.dropout = layers.Dropout(rate=0.5)

        # Output layer, class prediction.
        self.out = layers.Dense(num_classes)

    # Set forward pass.
    def call(self, x, is_training=False):
        x = tf.reshape(x, [-1, 64, 64, 1])
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.maxpool2(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.dropout(x, training=is_training)
        x = self.out(x)
        if not is_training:
            # tf cross entropy expect logits without softmax, so only
            # apply softmax when not training.
            x = tf.nn.softmax(x)
        return x

# Build neural network model.
conv_net = ConvNet()

In [23]:
# Define loss function
def cross_entropy_loss(x, y):
    # Convert labels to int 64 for tf cross-entropy function.
    y = tf.cast(y, tf.int64)
    # Apply softmax to logits and compute cross-entropy.
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=x)
    # Average loss across the batch.
    return tf.reduce_mean(loss)

# Accuracy metric
def accuracy(y_pred, y_true):
    # Predicted class is the index of highest score in prediction vector (i.e. argmax).
    correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))
    return tf.reduce_mean(tf.cast(correct_prediction, tf.float32), axis=-1)

# Stochastic gradient descent optimizer.
optimizer = tf.optimizers.Adam(learning_rate)

In [24]:
def run_optimization(x, y):
    # Wrap computation inside a GradientTape for automatic differentiation.
    with tf.GradientTape() as g:
        # Forward pass.
        pred = conv_net(x, is_training=True)
        # Compute loss.
        loss = cross_entropy_loss(pred, y)
        
    # Variables to update, i.e. trainable variables.
    trainable_variables = conv_net.trainable_variables

    # Compute gradients.
    gradients = g.gradient(loss, trainable_variables)
    
    # Update W and b following gradients.
    optimizer.apply_gradients(zip(gradients, trainable_variables))

In [14]:
# Optimization process. 
def run_optimization(x, y):
    # Wrap computation inside a GradientTape for automatic differentiation.
    with tf.GradientTape() as g:
        pred = logistic_regression(x)
        loss = cross_entropy(pred, y)

    # Compute gradients.
    gradients = g.gradient(loss, [W, b])
    
    # Update W and b following gradients.
    optimizer.apply_gradients(zip(gradients, [W, b]))

In [25]:
for step, (batch_x, batch_y) in enumerate(train_data.take(training_steps), 1):
    # Run the optimization to update W and b values.
    run_optimization(batch_x, batch_y)
    
    if step % display_step == 0:
        pred = conv_net(batch_x)
        loss = cross_entropy_loss(pred, batch_y)
        acc = accuracy(pred, batch_y)
        print("step: %i, loss: %f, accuracy: %f" % (step, loss, acc)) 

step: 10, loss: 2.302454, accuracy: 0.101562
step: 20, loss: 2.302723, accuracy: 0.085938
step: 30, loss: 2.302586, accuracy: 0.140625
step: 40, loss: 2.302596, accuracy: 0.117188
step: 50, loss: 2.302272, accuracy: 0.078125
step: 60, loss: 2.303321, accuracy: 0.085938
step: 70, loss: 2.301915, accuracy: 0.140625
step: 80, loss: 2.301997, accuracy: 0.117188
step: 90, loss: 2.302538, accuracy: 0.085938
step: 100, loss: 2.302555, accuracy: 0.117188
step: 110, loss: 2.302155, accuracy: 0.117188
step: 120, loss: 2.302248, accuracy: 0.093750
step: 130, loss: 2.302768, accuracy: 0.085938
step: 140, loss: 2.302821, accuracy: 0.101562
step: 150, loss: 2.302538, accuracy: 0.132812
step: 160, loss: 2.302855, accuracy: 0.070312
step: 170, loss: 2.302210, accuracy: 0.156250
step: 180, loss: 2.303019, accuracy: 0.070312
step: 190, loss: 2.302663, accuracy: 0.085938
step: 200, loss: 2.302607, accuracy: 0.132812


In [35]:
test_data = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_data = test_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)

In [44]:
acc = 0
test_steps = 200
for step, (batch_x, batch_y) in enumerate(test_data.take(test_steps), 1):
  pred = conv_net(batch_x)
  acc += accuracy(batch_x, batch_y)
print('Test accuracy : %f' % acc)

Test accuracy : 0.054688
